In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import os

# specify the URL of the website
url = "https://www.google.com/recaptcha/api2/demo"

# specify the number of times to repeat the process
num_repeats = 5

# create a new instance of the Chrome browser
driver = webdriver.Chrome()

# navigate to the website
driver.get(url)

# wait for reCAPTCHA iframe to load and switch to it
wait = WebDriverWait(driver, 10)
recaptcha_iframe = wait.until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, 'iframe[title="reCAPTCHA"]')))

# wait for div#rc-anchor-container to load and click on div.recaptcha-checkbox-border
recaptcha_wait = WebDriverWait(driver, 10)
recaptcha = recaptcha_wait.until(EC.presence_of_element_located((By.ID, 'rc-anchor-container')))
recaptcha.click()

# switch back to the main frame and wait for the "recaptcha challenge expires in two minutes" iframe to load
driver.switch_to.default_content()
challenge_iframe = wait.until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, 'iframe[title="recaptcha challenge expires in two minutes"]')))

# find the captcha image element and get its source
image = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'img[src*="api2/payload"]')))
image_src = image.get_attribute('src')

# download the image
output_folder = 'captchas/recaptcha'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
filename = 'captcha_1.png'
response = requests.get(image_src)
with open(os.path.join(output_folder, filename), "wb") as f:
    f.write(response.content)

# find the reload button and click it
reload_button = driver.find_element(By.ID,'recaptcha-reload-button')
reload_button.click()

# wait for the next image to load
wait.until(EC.staleness_of(image))

# download the next image
for i in range(2, 6):
    # find the captcha image element and get its source
    image = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'img[src*="api2/payload"]')))
    image_src = image.get_attribute('src')

    # download the image
    filename = f'captcha_{i}.png'
    response = requests.get(image_src)
    with open(os.path.join(output_folder, filename), "wb") as f:
        f.write(response.content)

    # find the reload button and click it
    reload_button = driver.find_element(By.ID,'recaptcha-reload-button')
    reload_button.click()

    # wait for the next image to load
    wait.until(EC.staleness_of(image))

# switch back to the main frame and quit the browser
driver.switch_to.default_content()
driver.quit()